In [1]:
import numpy as np
import pandas as pd

import torch 
import torch.nn as nn
import torch.optim
from torchvision import datasets, transforms

In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

device = 'cpu'
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    device = 'cuda'
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [3]:
torch.__version__

'1.8.1+cu101'

In [4]:
# Define a transform to transform from array to tensor
transform = transforms.Compose([transforms.ToTensor()])

# We will need to modify this to work with the hand gesture data, 
# this is a placeholder for now to test the code
train = datasets.MNIST('.', download=True, train=True, transform=transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
test = datasets.MNIST('.', download=True, train=False, transform=transform)

In [6]:
# Create DataLoader: Again, this will need to be modified to work with hand gesture data
trainloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

In [7]:
from torchvision.models import resnet18, vgg16, inception_v3, densenet121, resnet50

def get_model(model_name, num_classes):
  if model_name == "resnet18":
    model = resnet18(num_classes=num_classes)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) # need to modify the first layer to have 1 channel
  elif model_name == "resnet50":
    model = resnet50(num_classes=num_classes)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) # need to modify the first layer to have 1 channel
  elif model_name == "inception_v3":
    model = inception_v3(num_classes=num_classes)
    model.Conv2d_1a_3x3 = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False) # need to modify the first layer to have 1 channel
  return model


In [8]:
model = get_model("resnet18", 10) # Will need to change to 4
print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
def train_model(model, train_dl, optimizer, loss_fn, valid_dl = None, epochs=10):
  model.to(device)

  for ep in range(1, epochs + 1):
    train_loss, val_loss = [], []
    print("---" * 3, "Epoch", ep, "---" * 3)
    model.train()
    for img, lbl in train_dl:
      optimizer.zero_grad()
      img = img.to(device)
      out = model(img)
      loss = loss_fn(out, lbl.to(device))
      train_loss.append(loss.item())
      loss.backward()
      optimizer.step()
    print("Training Loss: ", np.mean(train_loss))
    if valid_dl is not None:
      model = model.eval()
      for img, lbl in valid_dl:
        img = img.to(device)
        out = model(img)
        loss = loss_fn(out, lbl.to(device))
        val_loss.append(loss.item())
      print("Validation Loss: ", np.mean(val_loss))

In [10]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
optimizer = torch.optim.Adam(model.parameters(), 0.005)
loss_fn = torch.nn.CrossEntropyLoss()

train_model(model, trainloader, optimizer, loss_fn, epochs=3)

--------- Epoch 1 ---------
Training Loss:  0.16490465357104367
--------- Epoch 2 ---------
Training Loss:  0.07124706767020382
--------- Epoch 3 ---------
Training Loss:  0.05399407194605269


In [12]:
def test_model(model, testloader):
  model.eval()
  y_true, y_pred = [], []
  for img, lbl in testloader:
    img = img.to(device)
    out = model(img)
    y_true.extend(lbl)
    probabilities = torch.softmax(model(img), dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    y_pred.extend(predicted_class.cpu())
  return y_true, y_pred

In [13]:
y_true, y_pred = test_model(model, testloader)
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.969     0.997     0.983       980
           1      0.988     0.997     0.993      1135
           2      0.977     0.991     0.984      1032
           3      0.982     0.989     0.986      1010
           4      0.990     0.987     0.988       982
           5      0.976     0.974     0.975       892
           6      0.994     0.979     0.986       958
           7      0.984     0.986     0.985      1028
           8      0.985     0.964     0.975       974
           9      0.984     0.960     0.972      1009

    accuracy                          0.983     10000
   macro avg      0.983     0.983     0.983     10000
weighted avg      0.983     0.983     0.983     10000

